In [55]:
# Importing numpy 
import numpy as np
# Importing Scipy 
import scipy as sp
# Importing Pandas Library 
import pandas as pd
# import glob function to scrap files path
from glob import glob
# import display() for better visualitions of DataFrames and arrays
from IPython.display import display
# import pyplot for plotting
import matplotlib.pyplot as plt
plt.style.use('bmh') # for better plots
import tqdm

# import data_loader for data loading
from data_loader import import_raw_signals, import_labels_file,normalize5,normalize2

from sklearn.metrics import accuracy_score

In [2]:
Raw_data_paths = sorted(glob("RawData/*"))
for fnams in Raw_data_paths:
    pass
Raw_acc_paths=Raw_data_paths[0:61]
Raw_gyro_paths=Raw_data_paths[61:122]

print (("RawData folder contains in total {:d} file ").format(len(Raw_data_paths)))
print (("The first {:d} are Acceleration files:").format(len(Raw_acc_paths)))
print (("The second {:d} are Gyroscope files:").format(len(Raw_gyro_paths)))
print ("The last file is a labels file")
print ("test labels file path is:",Raw_data_paths[122])
print ("train labels file path is:",Raw_data_paths[123])

RawData folder contains in total 124 file 
The first 61 are Acceleration files:
The second 61 are Gyroscope files:
The last file is a labels file
test labels file path is: RawData\label_test.txt
train labels file path is: RawData\label_train.txt


In [3]:
raw_dic={}
raw_acc_columns=['acc_X','acc_Y','acc_Z']
raw_gyro_columns=['gyro_X','gyro_Y','gyro_Z']
for path_index in range(0,61):
        key= Raw_data_paths[path_index][-16:-4]
        raw_acc_data_frame=import_raw_signals(Raw_data_paths[path_index],raw_acc_columns)
        raw_gyro_data_frame=import_raw_signals(Raw_data_paths[path_index+61],raw_gyro_columns)
        raw_signals_data_frame=pd.concat([raw_acc_data_frame, raw_gyro_data_frame], axis=1)
        raw_dic[key]=raw_signals_data_frame

In [4]:
print('raw_dic contains %d DataFrame' % len(raw_dic))
display(raw_dic['exp01_user01'].head(3))

raw_dic contains 61 DataFrame


,acc_X,acc_Y,acc_Z,gyro_X,gyro_Y,gyro_Z
0,0.918056,-0.112500,0.509722,-0.054978,-0.069639,-0.030849
1,0.911111,-0.093056,0.537500,-0.012523,0.019242,-0.038485
2,0.881944,-0.086111,0.513889,-0.023518,0.276417,0.006414


In [6]:
train_raw_labels_columns=['experiment_number_ID','user_number_ID','activity_number_ID','Label_start_point','Label_end_point']
test_raw_labels_columns=['experiment_number_ID','user_number_ID','Label_start_point','Label_end_point']

test_labels_path=Raw_data_paths[122]
train_labels_path=Raw_data_paths[123]

train_Labels_Data_Frame=import_labels_file(train_labels_path,train_raw_labels_columns)
test_Labels_Data_Frame=import_labels_file(test_labels_path,test_raw_labels_columns)

In [7]:
print ("The first 3 rows of  train_Labels_Data_Frame:" )
display(train_Labels_Data_Frame.head(3))
print(train_Labels_Data_Frame.shape)
display(test_Labels_Data_Frame.head(3))
print(test_Labels_Data_Frame.shape)

The first 3 rows of  train_Labels_Data_Frame:


,experiment_number_ID,user_number_ID,activity_number_ID,Label_start_point,Label_end_point
0,1,1,5,250,1232
1,1,1,4,1393,2194
2,1,1,5,2360,3374


(598, 5)


,experiment_number_ID,user_number_ID,Label_start_point,Label_end_point
0,3,2,298,1398
1,3,2,1686,2627
2,3,2,2770,3904


(258, 4)


In [8]:
from scipy.signal import medfilt

def median(signal):
    array=np.array(signal)   
    med_filtered=sp.signal.medfilt(array, kernel_size=3)
    return  med_filtered

In [9]:
from scipy.fftpack import fft  
from scipy.fftpack import fftfreq
from scipy.fftpack import ifft
import math 

sampling_freq = 50
nyq=sampling_freq/float(2)
freq1 = 0.3
freq2 = 20

# Function name: components_selection_one_signal

# Inputs: t_signal:1D numpy array (time domain signal); 

# Outputs: (total_component,t_DC_component , t_body_component, t_noise) 
#           type(1D array,1D array, 1D array)

# cases to discuss: if the t_signal is an acceleration signal then the t_DC_component is the gravity component [Grav_acc]
#                   if the t_signal is a gyro signal then the t_DC_component is not useful
# t_noise component is not useful
# if the t_signal is an acceleration signal then the t_body_component is the body's acceleration component [Body_acc]
# if the t_signal is a gyro signal then the t_body_component is the body's angular velocity component [Body_gyro]

def components_selection_one_signal(t_signal,freq1,freq2):
    t_signal=np.array(t_signal)
    t_signal_length=len(t_signal)
    f_signal=fft(t_signal)
    freqs=np.array(sp.fftpack.fftfreq(t_signal_length, d=1/float(sampling_freq)))# frequency values between [-25hz:+25hz]
    
    # DC_component: f_signal values having freq between [-0.3 hz to 0 hz] and from [0 hz to 0.3hz] 
    #                                                             (-0.3 and 0.3 are included)
    
    # noise components: f_signal values having freq between [-25 hz to 20 hz[ and from ] 20 hz to 25 hz] 
    #                                                               (-25 and 25 hz inculded 20hz and -20hz not included)
    
    # selecting body_component: f_signal values having freq between [-20 hz to -0.3 hz] and from [0.3 hz to 20 hz] 
    #                                                               (-0.3 and 0.3 not included , -20hz and 20 hz included)
    
    
    f_DC_signal=[] # DC_component in freq domain
    f_body_signal=[] # body component in freq domain numpy.append(a, a[0])
    f_noise_signal=[] # noise in freq domain
    
    for i in range(len(freqs)):# iterate over all available frequencies
        
        # selecting the frequency value
        freq=freqs[i]
        
        # selecting the f_signal value associated to freq
        value= f_signal[i]
        
        # Selecting DC_component values 
        if abs(freq)>0.3:# testing if freq is outside DC_component frequency ranges
            f_DC_signal.append(float(0)) # add 0 to  the  list if it was the case (the value should not be added)                                       
        else: # if freq is inside DC_component frequency ranges 
            f_DC_signal.append(value) # add f_signal value to f_DC_signal list
    
        # Selecting noise component values 
        if (abs(freq)<=20):# testing if freq is outside noise frequency ranges 
            f_noise_signal.append(float(0)) # # add 0 to  f_noise_signal list if it was the case 
        else:# if freq is inside noise frequency ranges 
            f_noise_signal.append(value) # add f_signal value to f_noise_signal

        # Selecting body_component values 
        if (abs(freq)<=0.3 or abs(freq)>20):# testing if freq is outside Body_component frequency ranges
            f_body_signal.append(float(0))# add 0 to  f_body_signal list
        else:# if freq is inside Body_component frequency ranges
            f_body_signal.append(value) # add f_signal value to f_body_signal list
    
    ################### Inverse the transformation of signals in freq domain ########################
    # applying the inverse fft(ifft) to signals in freq domain and put them in float format
    t_DC_component= ifft(np.array(f_DC_signal)).real
    t_body_component= ifft(np.array(f_body_signal)).real
    t_noise=ifft(np.array(f_noise_signal)).real
    
    total_component=t_signal-t_noise # extracting the total component(filtered from noise) 
                                     #  by substracting noise from t_signal (the original signal).
    
    # return outputs mentioned earlier
    return (total_component,t_DC_component,t_body_component,t_noise) 

In [10]:
import math
def mag_3_signals(x,y,z): # Euclidian magnitude
    return [math.sqrt((x[i]**2+y[i]**2+z[i]**2)) for i in range(len(x))]

In [11]:
dt=0.02 # dt=1/50=0.02s time duration between two rows
# Input: 1D array with lenght=N (N:unknown)
# Output: 1D array with lenght=N-1
def jerk_one_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [12]:
time_sig_dic={}
raw_dic_keys=sorted(raw_dic.keys())

for key in tqdm.tqdm(raw_dic_keys):
    raw_df=raw_dic[key]
    time_sig_df=pd.DataFrame()
    
    for column in raw_df.columns:
        t_signal=np.array(raw_df[column])
        med_filtred=median(t_signal)
        
        if 'acc' in column:
            _,grav_acc,body_acc,_=components_selection_one_signal(med_filtred,freq1,freq2)
            body_acc_jerk=jerk_one_signal(body_acc)
            time_sig_df['t_body_'+column]=body_acc[:-1]
            time_sig_df['t_grav_'+column]= grav_acc[:-1]
            time_sig_df['t_body_acc_jerk_'+column[-1]]=body_acc_jerk
        elif 'gyro' in column:
            _,_,body_gyro,_=components_selection_one_signal(med_filtred,freq1,freq2)
            body_gyro_jerk=jerk_one_signal(body_gyro)
            time_sig_df['t_body_gyro_'+column[-1]]=body_gyro[:-1]
            time_sig_df['t_body_gyro_jerk_'+column[-1]]=body_gyro_jerk
            
    new_columns_ordered=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                      't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                      't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z',
                      't_body_gyro_X','t_body_gyro_Y','t_body_gyro_Z',
                      't_body_gyro_jerk_X','t_body_gyro_jerk_Y','t_body_gyro_jerk_Z']
        
    ordered_time_sig_df=pd.DataFrame()
        
    for col in new_columns_ordered:
        ordered_time_sig_df[col]=time_sig_df[col]
        
    for i in range(0,15,3):
        mag_col_name=new_columns_ordered[i][:-1]+'mag'
        col0=np.array(ordered_time_sig_df[new_columns_ordered[i]]) # copy X_component
        col1=ordered_time_sig_df[new_columns_ordered[i+1]] # copy Y_component
        col2=ordered_time_sig_df[new_columns_ordered[i+2]] # copy Z_component
        mag_signal=mag_3_signals(col0,col1,col2)
        ordered_time_sig_df[mag_col_name]=mag_signal
        
    time_sig_dic[key]=ordered_time_sig_df

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [02:52<00:00,  2.83s/it]


In [13]:
display(time_sig_dic['exp01_user01'].shape)
display(time_sig_dic['exp01_user01'].describe())
time_sig_dic['exp01_user01'].head(3)

(20597, 20)

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag,t_body_gyro_mag,t_body_gyro_jerk_mag
count,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,2.059700e+04,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000,20597.000000
mean,0.000018,-0.000014,-0.000009,0.880358,-0.101501,0.097309,-0.001833,0.001183,0.000849,3.259619e-07,0.000008,0.000010,0.000046,0.000095,0.000362,0.165296,1.025479,3.404274,0.489705,9.439402
std,0.167505,0.123847,0.106477,0.299187,0.314444,0.262835,4.076298,3.237138,2.298076,3.907032e-01,0.497555,0.265791,7.641439,13.033311,5.724163,0.165557,0.025535,4.559251,0.480657,13.111758
min,-0.698870,-0.993486,-0.671478,-0.231975,-0.367943,-0.161622,-38.735217,-33.718078,-24.262993,-3.141414e+00,-2.694787,-4.050512,-90.662943,-158.457438,-89.662267,0.000864,0.768925,0.006144,0.001641,0.017986
25%,-0.044563,-0.024197,-0.042269,0.968003,-0.275034,-0.050273,-0.481136,-0.422376,-0.347403,-1.189116e-01,-0.139441,-0.059319,-1.100310,-1.705868,-0.832493,0.025437,1.021875,0.272200,0.087023,0.506674
50%,-0.001207,0.002857,-0.003154,0.997735,-0.236751,-0.025435,0.003867,0.003613,0.003452,3.737832e-03,-0.002389,0.003384,0.003147,-0.005965,-0.002999,0.114319,1.031724,1.581932,0.348679,4.510858
75%,0.020914,0.051177,0.026000,1.007358,-0.123642,0.090625,0.568717,0.505230,0.381417,1.546320e-01,0.101817,0.088021,1.159630,1.392834,0.799237,0.262351,1.038524,4.837894,0.758609,13.308692
max,1.031485,0.473201,0.493350,1.066684,0.832075,1.015052,34.647970,34.482683,35.302951,2.698454e+00,4.530312,2.260099,95.696959,151.910819,60.809206,1.180706,1.115731,54.209353,6.249451,160.671661


,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag,t_body_gyro_mag,t_body_gyro_jerk_mag
0,0.382899,-0.206562,-0.169117,0.431759,0.175976,0.720859,8.404087,-5.538872,-3.526643,-0.025624,-0.210093,-0.361419,0.552413,3.197618,-0.792801,0.466776,0.858500,10.665130,0.418831,3.340427
1,0.550980,-0.317339,-0.239649,0.443087,0.169277,0.715098,-9.140781,5.486163,3.704421,-0.014575,-0.146141,-0.377275,-0.185277,11.295307,2.384643,0.679496,0.858106,11.286035,0.404853,11.545770
2,0.368165,-0.207616,-0.165561,0.454376,0.162556,0.709293,3.968636,-2.902182,-2.103241,-0.018281,0.079766,-0.329582,-2.485754,6.100713,-0.485211,0.453938,0.857892,5.347556,0.339590,6.605536


In [17]:
def Windowing_type(time_sig_dic,Labels_Data_Frame):
    #import pdb;pdb.set_trace()
    columns=time_sig_dic['exp01_user01'].columns
    window_ID=0
    time_dictionary_window={}
    BA_array=np.array(Labels_Data_Frame)
    
    for line in tqdm.tqdm(BA_array):
        file_key= 'exp' + normalize2(int(line[0]))  +  '_user' + normalize2(int(line[1]))
        
        if line.shape[0] == 5 :
          act_ID=line[2]
          start_point=line[3]
          end_point = line[4]
        else :
          act_ID='None'
          start_point = line[2]
          end_point = line[3]
        
        for cursor in range(start_point,end_point-127,64):
            end_point=cursor+128
            data=np.array(time_sig_dic[file_key].iloc[cursor:end_point])
            window=pd.DataFrame(data=data,columns=columns)
            key='t_W'+normalize5(window_ID)+'_'+file_key+'_act'+normalize2(act_ID)
            time_dictionary_window[key]=window
            window_ID=window_ID+1
    
    return time_dictionary_window 

In [18]:
train_time_dictionary_window  = Windowing_type(time_sig_dic,train_Labels_Data_Frame)
test_time_dictionary_window  = Windowing_type(time_sig_dic,test_Labels_Data_Frame)

100%|███████████████████████████████████████████████████████████████████████████████| 258/258 [00:02<00:00, 121.38it/s]


In [19]:
train_window = train_time_dictionary_window[sorted(train_time_dictionary_window.keys())[0]]
train_window.head()

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_gyro_X,t_body_gyro_Y,t_body_gyro_Z,t_body_gyro_jerk_X,t_body_gyro_jerk_Y,t_body_gyro_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag,t_body_gyro_mag,t_body_gyro_jerk_mag
0,-0.003068,0.009275,-0.049713,1.025297,-0.135052,0.153468,-0.064668,0.067917,-0.010441,0.023431,0.089860,0.049693,-0.193282,-0.501977,0.204316,0.050664,1.045479,0.094359,0.105324,0.575399
1,-0.004362,0.010634,-0.049921,1.025123,-0.135542,0.152980,-0.044544,0.049300,0.188938,0.019565,0.079820,0.053780,0.630392,0.991992,-0.376632,0.051227,1.045300,0.200280,0.098216,1.234218
2,-0.005253,0.011620,-0.046143,1.024940,-0.136014,0.152417,0.021576,-0.060893,0.020793,0.032173,0.099660,0.046247,-0.058003,-0.047758,-0.215589,0.047872,1.045099,0.067866,0.114481,0.228307
3,-0.004821,0.010402,-0.045727,1.024747,-0.136467,0.151784,-0.178322,-0.007597,0.229162,0.031013,0.098705,0.041935,-0.260247,-0.011292,0.465103,0.047142,1.044877,0.290469,0.111638,0.533083
4,-0.008388,0.010250,-0.041144,1.024545,-0.136900,0.151080,0.168810,-0.075081,-0.268611,0.025808,0.098479,0.051237,0.188867,0.892604,-0.057455,0.043223,1.044634,0.326015,0.113971,0.914174


In [20]:
print("시간 도메인 Train 데이터 수 : {}".format(len(train_time_dictionary_window)))
print("시간 도메인 Test 데이터 수 : {}".format(len(test_time_dictionary_window)))
print("윈도우 크기(2.56s => 128개) : {}".format(len(train_window)))

시간 도메인 Train 데이터 수 : 7283
시간 도메인 Test 데이터 수 : 3116
윈도우 크기(2.56s => 128개) : 128


In [21]:
from scipy import fftpack
from numpy.fft import *

def fast_fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    
    return amplitude_f_signal

def fast_fourier_transform(t_window):
    f_window=pd.DataFrame()
    for column in t_window.columns:
        if 'grav' not in column:
            t_signal=np.array(t_window[column])
            f_signal= np.apply_along_axis(fast_fourier_transform_one_signal,0,t_signal)
            f_window["f_"+column[2:]]=f_signal
    return f_window

In [22]:
train_frequent_dictionary_window = {'f'+key[1:] : train_t_df.pipe(fast_fourier_transform) for key, train_t_df in tqdm.tqdm(train_time_dictionary_window.items())}
test_frequent_dictionary_window = {'f'+key[1:] : test_t_df.pipe(fast_fourier_transform) for key, test_t_df in tqdm.tqdm(test_time_dictionary_window.items())}

100%|█████████████████████████████████████████████████████████████████████████████| 3116/3116 [00:24<00:00, 128.64it/s]


In [23]:
train_window = train_frequent_dictionary_window[sorted(train_frequent_dictionary_window.keys())[0]]
train_window.head()

,f_body_acc_X,f_body_acc_Y,f_body_acc_Z,f_body_acc_jerk_X,f_body_acc_jerk_Y,f_body_acc_jerk_Z,f_body_gyro_X,f_body_gyro_Y,f_body_gyro_Z,f_body_gyro_jerk_X,f_body_gyro_jerk_Y,f_body_gyro_jerk_Z,f_body_acc_mag,f_body_acc_jerk_mag,f_body_gyro_mag,f_body_gyro_jerk_mag
0,0.257558,0.055128,0.568459,0.004474,1.018802,2.057622,1.302252,2.880254,0.066856,0.574643,8.573938,3.442790,3.790846,25.481237,14.048363,73.764900
1,0.324072,1.169116,2.325911,0.794786,1.852232,4.739415,2.270811,8.786654,3.040972,5.543606,13.054902,4.222888,0.110791,1.628419,2.657498,6.699360
2,0.045438,0.300832,0.599853,0.220647,0.463545,1.179457,0.332364,2.154464,0.845314,1.462829,2.404663,0.720631,0.721155,0.921396,1.619180,3.680039
3,0.026573,0.162318,0.341578,0.195753,0.523164,0.458591,0.396876,1.331866,0.486980,2.359180,1.583174,0.965404,0.326605,1.490716,0.289519,2.877012
4,0.040355,0.192653,0.218084,0.391329,0.872601,1.024741,0.484334,0.835291,0.432675,4.736053,0.525072,1.014540,0.252132,0.395907,0.434509,0.690402


In [24]:
print("주파수 도메인 Train 데이터 수 : {}".format(len(train_frequent_dictionary_window)))
print("주파수 도메인 Test 데이터 수 : {}".format(len(test_frequent_dictionary_window)))
print("피처의 갯수 : {}".format(len(train_window)))

주파수 도메인 Train 데이터 수 : 7283
주파수 도메인 Test 데이터 수 : 3116
피처의 갯수 : 128


In [39]:
# -------------------------------------
# [Empty Module #1] Feature Engineering
# -------------------------------------

# -------------------------------------
# Feature Engineering
# -------------------------------------
# 목적: 제공된 36개의 시퀀스 도메인 데이터를 기반으로 유의미한 피처를 추출한다.
# 입력인자: 시간(time) 도메인 Feature 20개 , 주파수(frequency) 도메인 Feature 16개
# 출력인자: 분류모델 학습을 위한 Feature
# -------------------------------------

# ------------------------------------------------------------
# 구현 가이드라인 - 논문에서 제안하는 Feature Engineering 방법
# ------------------------------------------------------------
#
# mean(): Mean value
# std(): Standard deviation
# mad(): Median absolute deviation 
# max(): Largest value in array
# min(): Smallest value in array
# sma(): Signal magnitude area
# energy(): Energy measure. Sum of the squares divided by the number of values. 
# iqr(): Interquartile range 
# entropy(): Signal entropy
# arCoeff(): Autorregresion coefficients with Burg order equal to 4
# correlation(): correlation coefficient between two signals
# maxInds(): index of the frequency component with largest magnitude
# meanFreq(): Weighted average of the frequency components to obtain a mean frequency
# skewness(): skewness of the frequency domain signal 
# kurtosis(): kurtosis of the frequency domain signal 
# bandsEnergy(): Energy of a frequency interval within the 64 bins of the FFT of each window.
# angle(): Angle between to vectors.

import sys

# Time domain Feature Extract function

from Feature_engineering import mean_axial,std_axial,mad_axial,max_axial,min_axial, t_sma_axial, t_energy_axial,IQR_axial,entropy_axial, t_arburg_axial, t_corr_axial
from Feature_engineering import mean_mag,std_mag,mad_mag,max_mag,min_mag,t_sma_mag,t_energy_mag,IQR_mag,entropy_mag,t_arburg_mag

# Frequency domain Feature Extract function
from Feature_engineering import f_sma_axial,f_energy_axial,f_max_Inds_axial,f_mean_Freq_axial,f_skewness_and_kurtosis_axial,f_all_bands_energy_axial
from Feature_engineering import f_sma_mag,f_energy_mag,f_max_Inds_mag,f_mean_Freq_mag,f_skewness_mag,f_kurtosis_mag

# Additional Feature Extract function
from Feature_engineering import angle_features

def feature_extractor(time_dictionary,freq_dictionary, condition='train') :
    
    
    if condition == 'train' :
        total_data = []
        total_label = []
    elif condition == 'test' :
        total_data = []
        
    for i in tqdm.tqdm(range(len(time_dictionary))) :
        
        time_key = sorted(time_dictionary.keys())[i]
        freq_key = sorted(freq_dictionary.keys())[i]
        
        time_window = time_dictionary[time_key]
        freq_window = freq_dictionary[freq_key]
        
        if condition == 'train' :
          window_user_id= int(time_key[-8:-6]) # extract the user id from window's key
          window_activity_id=int(time_key[-2:]) # extract the activity id from the windows key
        elif condition == 'test' :
          window_user_id= int(time_key[-10:-8]) # extract the user id from window's key
          window_activity_id= 0
        else :
            print("Error")
            sys.exit()
            break;
            
        ##################################################################################
        
        
        # Time domain - Feature extractor - Part 1. axial(X,Y,Z) Features 
        
        #[0,1,2] : 't_body_acc_X', 't_body_acc_Y', 't_body_acc_Z'
        #[3,4,5] : 't_grav_acc_X','t_grav_acc_Y', 't_grav_acc_Z'
        #[6,7,8] : 't_body_acc_jerk_X','t_body_acc_jerk_Y', 't_body_acc_jerk_Z'
        #[9,10,11] : 't_body_gyro_X','t_body_gyro_Y', 't_body_gyro_Z'
        #[12,13,14] : 't_body_gyro_jerk_X', 't_body_gyro_jerk_Y', 't_body_gyro_jerk_Z'
        
        axial_columns = time_window.columns[0:15]
        axial_df = time_window[axial_columns] # X,Y,Z
        
        time_axial_features = []
        
        for col in range(0,15,3) : 
            
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 time_3axial_vector 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            mean_vector = mean_axial(axial_df.iloc[:,col:col+3])
            std_vector = std_axial(axial_df.iloc[:,col:col+3])
            mad_vector = mad_axial(axial_df.iloc[:,col:col+3])
            max_vector = max_axial(axial_df.iloc[:,col:col+3])
            min_vector = min_axial(axial_df.iloc[:,col:col+3])
            sma_value = t_sma_axial(axial_df.iloc[:,col:col+3])
            energy_vector = t_energy_axial(axial_df.iloc[:,col:col+3])
            IQR_vector = IQR_axial(axial_df.iloc[:,col:col+3])
            entropy_vector = entropy_axial(axial_df.iloc[:,col:col+3])
            AR_vector = t_arburg_axial(axial_df.iloc[:,col:col+3])
            corr_vector = t_corr_axial(axial_df.iloc[:,col:col+3])
            
            # 40 value per each 3-axial signals
            time_3axial_vector= mean_vector + std_vector + mad_vector + max_vector + min_vector + [sma_value] + energy_vector + IQR_vector + entropy_vector + AR_vector + corr_vector
            
            # append these features to the global list of features
            time_axial_features= time_axial_features+ time_3axial_vector
        
        ##################################################################################
        
        # Time domain - Feature extractor - Part 2. Magnitude Features 
        
        #[15]'t_body_acc_mag'
        #[16]'t_grav_acc_mag'
        #[17]'t_body_acc_jerk_mag'
        #[18]'t_body_gyro_mag'
        #[19]'t_body_gyro_jerk_mag'
        
        mag_columns = time_window.columns[15:]
        mag_columns = time_window[mag_columns]
        
        time_mag_features = []
        
        for col in mag_columns :
            
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 col_mag_values 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            mean_value = mean_mag(time_window[col])
            std_value = std_mag(time_window[col])
            mad_value = mad_mag(time_window[col])
            max_value = max_mag(time_window[col])
            min_value = min_mag(time_window[col])
            sma_value = t_sma_mag(time_window[col])
            energy_value = t_energy_mag(time_window[col])
            IQR_value = IQR_mag(time_window[col])
            entropy_value = entropy_mag(time_window[col])
            AR_vector = t_arburg_mag(time_window[col])
            # 13 value per each t_mag_column
            col_mag_values = [mean_value, std_value, mad_value, max_value, min_value, sma_value, 
                              energy_value,IQR_value, entropy_value]+ AR_vector

            # col_mag_values will be added to the global list
            time_mag_features= time_mag_features+ col_mag_values

        
        ##################################################################################
        
        # Frequency domain - Feature extractor - Part 1. axial(X,Y,Z) Features 
        
        #[0,1,2] : 'f_body_acc_X', 'f_body_acc_Y', 'f_body_acc_Z'
        #[3,4,5] : 'f_body_acc_jerk_X','f_body_acc_jerk_Y', 'f_body_acc_jerk_Z'
        #[6,7,8] : 'f_body_gyro_X','f_body_gyro_Y', 'f_body_gyro_Z'
        #[9,10,11] : 'f_body_gyro_jerk_X','f_body_gyro_jerk_Y', 'f_body_gyro_jerk_Z'
        
        axial_columns=freq_window.columns[0:12]
        axial_df=freq_window[axial_columns]
        freq_axial_features=[]
        
        for col in range(0,12,3) :         
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 freq_3axial_features 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            mean_vector = mean_axial(axial_df.iloc[:,col:col+3])
            std_vector = std_axial(axial_df.iloc[:,col:col+3])
            mad_vector = mad_axial(axial_df.iloc[:,col:col+3])
            max_vector = max_axial(axial_df.iloc[:,col:col+3])
            min_vector = min_axial(axial_df.iloc[:,col:col+3])
            sma_value = f_sma_axial(axial_df.iloc[:,col:col+3])
            energy_vector = f_energy_axial(axial_df.iloc[:,col:col+3])
            IQR_vector = IQR_axial(axial_df.iloc[:,col:col+3])
            entropy_vector = entropy_axial(axial_df.iloc[:,col:col+3])
            max_inds_vector = f_max_Inds_axial(axial_df.iloc[:,col:col+3])
            mean_Freq_vector =  f_mean_Freq_axial(axial_df.iloc[:,col:col+3])
            skewness_and_kurtosis_vector = f_skewness_and_kurtosis_axial(axial_df.iloc[:,col:col+3])
            bands_energy_vector= f_all_bands_energy_axial(axial_df.iloc[:,col:col+3])
            
            freq_3axial_features = mean_vector +std_vector + mad_vector + max_vector + min_vector + [sma_value] + energy_vector + IQR_vector + entropy_vector + max_inds_vector + mean_Freq_vector + skewness_and_kurtosis_vector + bands_energy_vector
            freq_axial_features = freq_axial_features+ freq_3axial_features
        
        ##################################################################################
        
        # Frequency domain - Feature extractor - Part 2. Magnitude Features
        
        #[12]'f_body_acc_mag'
        #[13]'f_body_acc_jerk_mag'
        #[14]'f_body_gyro_mag'
        #[15]'f_body_gyro_jerk_mag'
        
        mag_columns=freq_window.columns[12:]
        mag_columns=freq_window[mag_columns]
        
        freq_mag_features = []
        
        for col in mag_columns:
            # ------------------------------------------------------------
            # 구현 가이드라인 
            # ------------------------------------------------------------
            # 아래 col_mag_values에 나타난 Feature를 계산하여야 한다.
            # 각각을 계산하기위한 함수는 'Feature_engineering.py'에 내제되어 있다.
            # ------------------------------------------------------------
            mean_value = mean_mag(freq_window[col])
            std_value = std_mag(freq_window[col])
            mad_value = mad_mag(freq_window[col])
            max_value = max_mag(freq_window[col])
            min_value = min_mag(freq_window[col])
            sma_value = f_sma_mag(freq_window[col])
            energy_value = f_energy_mag(freq_window[col])
            IQR_value = IQR_mag(freq_window[col])
            entropy_value = entropy_mag(freq_window[col])
            max_Inds_value = f_max_Inds_mag(freq_window[col])
            mean_Freq_value = f_mean_Freq_mag(freq_window[col])
            skewness_value = f_skewness_mag(freq_window[col])
            kurtosis_value = f_kurtosis_mag(freq_window[col])
            # 13 value per each t_mag_column
            col_mag_values = [mean_value, std_value, mad_value, max_value, 
                              min_value, sma_value, energy_value,IQR_value, 
                              entropy_value, max_Inds_value, mean_Freq_value,
                              skewness_value, kurtosis_value ]
            
            freq_mag_features= freq_mag_features+ col_mag_values
        
        ##################################################################################
        
        # Time domain - Feature extractor - Part 3. Additional Features 
        
        additional_features = angle_features(time_window)
                
        ##################################################################################
        
        total_features = time_axial_features + time_mag_features + freq_axial_features + freq_mag_features + additional_features
        
        total_data.append(total_features)
        if condition == 'train' :
            total_label.append(window_activity_id)
    
    total_data = np.array(total_data)
    if condition == 'train' :
        total_label = np.array(total_label)
    
    if condition == 'train' :
        return total_data, total_label
    elif condition == 'test' :
        return total_data

In [40]:
train_data, train_label = feature_extractor(train_time_dictionary_window,train_frequent_dictionary_window,condition='train')
test_data = feature_extractor(test_time_dictionary_window,test_frequent_dictionary_window,condition='test')

100%|██████████████████████████████████████████████████████████████████████████████| 3116/3116 [02:19<00:00, 22.27it/s]


In [46]:
# -------------------------------------
# [Empty Module #2] Data Normalization
# -------------------------------------

# -------------------------------------
# Data Normalization
# -------------------------------------
# 목적: 앞서 구축한 train,test 셋에 대한 Feature를 정규화한다.
# 입력인자: train 셋에서 추출된 Feature, test 셋에서 추출된 Feature
# 출력인자: 정규화된 Feature Vector
# -------------------------------------


from sklearn.preprocessing import MinMaxScaler

# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# sklearn에서 제공하는 MinMaxScaler를 사용해 데이터 정규화를 진행한다.
# (MinMaxScaler가 아닌 다른 정규화를 사용할 수 있다.)
# ------------------------------------------------------------
sc= MinMaxScaler()
sc.fit(train_data)
train_data = sc.transform(train_data)
test_data = sc.transform(test_data)

In [65]:
# -------------------------------------
# [Empty Module #3] RandomForest를 이용한 분류
# -------------------------------------

# -------------------------------------
# RandomForest를 이용한 분류
# -------------------------------------
# 목적: 앞서 완성한 train/test Feature를 RandomForest를 이용해 분류한다.
# 추가 안내 : 리더보드의 베이스라인은 random_state를 0으로 하였다.
# 입력인자: Feature vector(train/test)
# 출력인자: 분류결과
# -------------------------------------

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# sklearn에서 제공하는 RandomForest를 사용해 분류를 진행한다.
# (RandomForest를가 아닌 다른 분류모델을 사용할 수 있다.)
# ------------------------------------------------------------
model=RandomForestClassifier(max_depth = 20, random_state = 0)
model2= DecisionTreeClassifier(max_depth = 20, random_state = 0)
model.fit(train_data, train_label)
model2.fit(train_data, train_label)
train_label_predict = model.predict(train_data)
train_label_predict2 = model2.predict(train_data)
print(accuracy_score(train_label, train_label_predict))
print(accuracy_score(train_label, train_label_predict2))
y_pred = model.predict(test_data)

1.0
0.9994507757792118


In [66]:
submit_csv = pd.read_csv('submission.csv')
submit_csv['Label'] = y_pred
submit_csv['Label'] = submit_csv['Label'].astype("int")
submit_csv.to_csv("submission.csv", index=False)